<a href="https://colab.research.google.com/github/BradHolden415/Royce-Project_Data-Construct/blob/main/Data_Construct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# give Google Colab access to Google Drive folders
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing the main database
df = pd.read_csv('/content/drive/MyDrive/4 WORK/CLP/Databases/database.csv')

<ipython-input-5-6236e8e5facf>:2: DtypeWarning: Columns (2,3,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/4 WORK/CLP/Databases/database.csv')


In [ ]:
# importing the revenue data and storing it as a pandas dataframe
rev_filename = "/content/drive/MyDrive/4 WORK/CLP/Databases/revenue.csv"
df_rev = pd.read_csv(rev_filename)

In [ ]:
# rename the columns of the dataframe to make them more readable
df_rev.rename(columns={'District Number': 'LEA_num', 'District Name': 'LEA', 'Location School Id':
                           'school_ID', 'Location School Number': 'school_num', 'Location Name': 'school',
                           'Fund Code': 'fund', 'Revenue Code': 'function', 'Program Code': 'program',
                           'Amount': 'amt', 'Year': 'year'}, inplace=True)

# remove rows from revenue dataframe that include missing data
df_rev.dropna(axis=0, how='all', inplace=True)

# add a column called "revenue" which designates that each fund listed is a revenue fund, not an expense fund
df_rev['revenue'] = True

In [ ]:
df_rev.head()

,LEA_num,LEA,school_ID,school_num,school,fund,function,Revenue Source Two Digit,program,amt,year,n,Unnamed: 12,revenue
0,001,Alpine District,122.0,001,Alpine District,10.0,1110.0,1000.0,5.0,30714587.17,2016.0,1.0,Y,True
1,001,Alpine District,122.0,001,Alpine District,10.0,1111.0,1000.0,5.0,1897475.78,2016.0,2.0,Y,True
2,001,Alpine District,122.0,001,Alpine District,10.0,1112.0,1000.0,5.0,23690571.60,2016.0,3.0,Y,True
3,001,Alpine District,122.0,001,Alpine District,10.0,1113.0,1000.0,5.0,1463548.44,2016.0,4.0,Y,True
4,001,Alpine District,122.0,001,Alpine District,10.0,1114.0,1000.0,5.0,12062125.79,2016.0,5.0,Y,True


In [ ]:
# remove irrelevant columns
df_rev.drop(["LEA_num", "Revenue Source Two Digit", "Unnamed: 12", "n"], axis=1, inplace=True)

In [ ]:
# import expense data, perform the same steps from revenue data
exp_filename = "/content/drive/MyDrive/4 WORK/CLP/Databases/expense.csv"
df_exp = pd.read_csv(exp_filename)

<ipython-input-11-25572c8aff05>:3: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_exp = pd.read_csv(exp_filename)


In [ ]:
df_exp.rename(columns={'District Number': 'LEA_num', 'District Name': 'LEA', 'Location School Id':
    'school_ID', 'Location School Number': 'school_num', 'Location Name': 'school',
                       'Fund Code': 'fund', 'Function Code': 'function', 'Program Code': 'program',
                       'Amount': 'amt', 'Year': 'year'}, inplace=True)

df_exp.dropna(axis=0, how='all', inplace=True)

df_exp['revenue'] = False

In [ ]:
df_exp.head()

,LEA_num,LEA,school_ID,school_num,school,fund,function,Object Code,program,amt,year,revenue
0,001,Alpine District,122,001,Alpine District,10,1000,131,5.0,1.155745e+08,2016,False
1,001,Alpine District,122,001,Alpine District,10,1000,131,1205.0,1.301380e+07,2016,False
2,001,Alpine District,122,001,Alpine District,10,1000,131,1210.0,6.916244e+05,2016,False
3,001,Alpine District,122,001,Alpine District,10,1000,131,1278.0,2.964000e+05,2016,False
4,001,Alpine District,122,001,Alpine District,10,1000,131,5331.0,5.098589e+04,2016,False


In [ ]:
df_exp.drop(['LEA_num', 'Object Code'], axis = 1, inplace=True)

In [ ]:
# combine both the expense and revenue datasets together
df = df_rev.append(df_exp)

# convert each string into ALL CAPS to prevent issues with key/value mapping
df = df.applymap(lambda s: s.upper() if type(s) == str else s)

<ipython-input-15-311bb6833e32>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_rev.append(df_exp)


In [ ]:
# import a conversion table between LEA names and their respective ID values
LEA_keys = "/content/drive/MyDrive/4 WORK/CLP/Databases/district_keys.csv"
df_LEA_keys = pd.read_csv(LEA_keys)

In [ ]:
# convert ID values to integers to remove trailing zeroes
df_LEA_keys['ID'] = df_LEA_keys['ID'].astype('int').astype('str')
df_LEA_keys = df_LEA_keys.applymap(lambda s: s.upper() if type(s) == str else s)

In [ ]:
# convert each string into ALL CAPS to prevent issues with key/value mapping
df_LEA_keys = pd.Series(df_LEA_keys['ID'].values, index=df_LEA_keys['LEA']).to_dict()

In [ ]:
# using the new conversion table, add LEA_ID to the expense/revenue dataframe
df['LEA_ID'] = df['LEA']
df['LEA_ID'] = df['LEA_ID'].map(df_LEA_keys)

In [ ]:
# do the same conversions with school names to school IDs
school_keys = "/content/drive/MyDrive/4 WORK/CLP/Databases/school_keys.csv"
df_school_keys = pd.read_csv(school_keys).astype(str)
df_school_keys = df_school_keys.applymap(lambda s: s.upper())

In [ ]:
df_school_keys = pd.Series(df_school_keys['ID'].values, index=df_school_keys['School']).to_dict()

In [ ]:
df['school_ID'] = df['school']
df['school_ID'] = df['school'].map(df_school_keys)

In [ ]:
# export the financial data to 'financial.csv'
df.to_csv('financial.csv')

In [ ]:
# identify the percentage of null school IDs
df['school_ID'].isnull().sum() * 100 / len(df['school_ID'])

0.28985734933702006

In [ ]:
# read in RISE proficiency data into a pandas dataframe
df_RISE = pd.read_csv('/content/drive/MyDrive/4 WORK/CLP/Databases/rise.csv')

# rename columns for readability
df_RISE = df_RISE[['year', 'test', 'school',
       'LEA', 'LEA_ID', 'subject',
       'proficiency']]

# remove missing rows
df_RISE.dropna(axis=0, how='all', inplace=True)

# convert datatypes
conversions = {'year': int, 'test': str, 'school': str, 'LEA': str, 'LEA_ID': str, 'subject': str, 'proficiency': float}
df_RISE = df_RISE.astype(conversions)

# convert each string into ALL CAPS to prevent issues with key/value mapping
df_RISE = df_RISE.applymap(lambda s: s.upper() if type(s) == str else s)

In [ ]:
# read in the SAGE data and perform the same conversions
df_SAGE = pd.read_csv('/content/drive/MyDrive/4 WORK/CLP/Databases/sage.csv')
df_SAGE.dropna(axis=0, how='all', inplace=True)
conversions = {'year': int, 'test': str, 'school': str, 'LEA': str, 'subject': str, 'proficiency': float}
df_SAGE = df_SAGE.astype(conversions)
df_SAGE = df_SAGE.applymap(lambda s: s.upper() if type(s) == str else s)

In [ ]:
# map the school and LEA names to their respective IDs from the conversion tables we used earlier
df_SAGE['school_ID'] = df_SAGE['school']
df_SAGE['school_ID'] = df_SAGE['school'].map(df_school_keys)

df_RISE['school_ID'] = df_RISE['school']
df_RISE['school_ID'] = df_RISE['school'].map(df_school_keys)

df_SAGE['LEA_ID'] = df_SAGE['LEA']
df_SAGE['LEA_ID'] = df_SAGE['LEA'].map(df_LEA_keys)

df_RISE['LEA_ID'] = df_RISE['LEA']
df_RISE['LEA_ID'] = df_RISE['LEA'].map(df_LEA_keys)

In [ ]:
# identify the percent of missing school/LEA IDs and names
print(df_SAGE['school_ID'].isnull().sum() * 100 / len(df_SAGE['school_ID']))
print(df_RISE['school_ID'].isnull().sum() * 100 / len(df_RISE['school_ID']))

print(df_SAGE['LEA_ID'].isnull().sum() * 100 / len(df_SAGE['LEA_ID']))
print(df_RISE['LEA_ID'].isnull().sum() * 100 / len(df_RISE['LEA_ID']))

1.3778643103190056
0.1979544704717915
0.3504567919724427
0.0


In [ ]:
# aggregate the data for each LEA with subjects as different columns so that we have total LEA proficiency data to match with revenue/expense data at the LEA level
df_SAGE_agg = df_SAGE.groupby(['LEA', 'year', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()
df_SAGE_agg['school'] = df_SAGE_agg['LEA']
df_SAGE_agg['school_ID'] = df_SAGE_agg['school'].map(df_school_keys)
df_RISE_agg = df_RISE.groupby(['LEA', 'year', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()
df_RISE_agg['school'] = df_RISE_agg['LEA']
df_RISE_agg['school_ID'] = df_RISE_agg['school'].map(df_school_keys)

In [ ]:
df_RISE_agg

subject,LEA,year,LEA_ID,test,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,school,school_ID
0,ACADEMY FOR MATH ENGINEERING & SCIENCE,2019,110575,RISE,0.543900,0.386300,NaN,ACADEMY FOR MATH ENGINEERING & SCIENCE,700
1,ACADEMY FOR MATH ENGINEERING & SCIENCE,2021,110575,RISE,0.678161,0.550000,0.505376,ACADEMY FOR MATH ENGINEERING & SCIENCE,700
2,ADVANTAGE ARTS ACADEMY,2021,186506,RISE,0.290780,0.223022,0.337209,ADVANTAGE ARTS ACADEMY,01M
3,ALPINE DISTRICT,2019,122,RISE,0.343374,0.283145,0.303801,ALPINE DISTRICT,1
4,ALPINE DISTRICT,2021,122,RISE,0.496344,0.456673,0.505752,ALPINE DISTRICT,1
...,...,...,...,...,...,...,...,...,...
286,WAYNE DISTRICT,2021,1042,RISE,0.420071,0.406880,0.392804,WAYNE DISTRICT,34
287,WEBER DISTRICT,2019,1050,RISE,0.285675,0.245945,0.253145,WEBER DISTRICT,35
288,WEBER DISTRICT,2021,1050,RISE,0.359681,0.374155,0.378302,WEBER DISTRICT,35
289,WEILENMANN SCHOOL OF DISCOVERY,2019,186157,RISE,0.355000,0.288000,0.319700,WEILENMANN SCHOOL OF DISCOVERY,100


In [ ]:
# convert the data into a long format to merge with expense/revenue data
df_SAGE_unstack = df_SAGE.groupby(['LEA', 'school', 'year', 'school_ID', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()
df_RISE_unstack = df_RISE.groupby(['LEA', 'school', 'year', 'school_ID', 'LEA_ID', 'subject', 'test'])['proficiency'].mean().unstack('subject').reset_index()

In [ ]:
# combine all of the proficiency data together
df_prof = df_RISE_unstack.append(df_SAGE_unstack)
df_prof = df_prof.append(df_SAGE_agg)
df_prof = df_prof.append(df_RISE_agg)

<ipython-input-34-42022fc7a805>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prof = df_RISE_unstack.append(df_SAGE_unstack)
<ipython-input-34-42022fc7a805>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prof = df_prof.append(df_SAGE_agg)
<ipython-input-34-42022fc7a805>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_prof = df_prof.append(df_RISE_agg)


In [ ]:
df_prof['stdev'] = df_prof[['MATHEMATICS', 'ENGLISH LANGUAGE ARTS', 'SCIENCE']].std(axis=1)

In [ ]:
df_prof

subject,LEA,school,year,school_ID,LEA_ID,test,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,stdev
0,ACADEMY FOR MATH ENGINEERING & SCIENCE,ACADEMY FOR MATH ENGINEERING & SCIENCE,2019,700,110575,RISE,0.543900,0.386300,NaN,0.111440
1,ACADEMY FOR MATH ENGINEERING & SCIENCE,ACADEMY FOR MATH ENGINEERING & SCIENCE,2021,700,110575,RISE,0.678161,0.550000,0.505376,0.089695
2,ADVANTAGE ARTS ACADEMY,ADVANTAGE ARTS ACADEMY,2021,01M,186506,RISE,0.290780,0.223022,0.337209,0.057425
3,ALPINE DISTRICT,ALPINE ONLINE SCHOOL,2019,299,122,RISE,0.304300,0.355600,NaN,0.036275
4,ALPINE DISTRICT,ALPINE ONLINE SCHOOL,2021,299,122,RISE,0.477124,0.364780,0.583333,0.109291
...,...,...,...,...,...,...,...,...,...,...
286,WAYNE DISTRICT,WAYNE DISTRICT,2021,34,1042,RISE,0.420071,0.406880,0.392804,0.013636
287,WEBER DISTRICT,WEBER DISTRICT,2019,35,1050,RISE,0.285675,0.245945,0.253145,0.021168
288,WEBER DISTRICT,WEBER DISTRICT,2021,35,1050,RISE,0.359681,0.374155,0.378302,0.009776
289,WEILENMANN SCHOOL OF DISCOVERY,WEILENMANN SCHOOL OF DISCOVERY,2019,100,186157,RISE,0.355000,0.288000,0.319700,0.033516


## We lost the demographics database. We are currently waiting to see if Grayson has the demographic.csv in his possesion. If he doesn't we will have to create a new one.

In [ ]:
# read in the demographic data and store as pandas dataframe
df_demo = pd.read_csv('/content/drive/MyDrive/Clear Learning Partners/Database/royce/demographic.csv')

# convert datatypes
conversions = {'year': int, 'type': str, 'LEA': str, 'school': str, 'k_12': int, 'k': int, 'grade_1': int, 'grade_2': int, 'grade_3': int, 'grade_4': int, 'grade_5': int, 'grade_6': int,
               'grade_7': int, 'grade_8': int, 'grade_9': int, 'grade_10': int, 'grade_11': int, 'grade_12': int, 'female': int, 'male': int, 'american_indian': int,
               'afam_black': int, 'asian': int, 'hispanic': int, 'multi_race': int, 'pacific_islander': int, 'white': int, 'econ_disadvantaged': int, 'ELL': int, 'disabled': int,
               'homeless': int, 'preschool': int}
df_demo = df_demo.astype(conversions)

# convert each string into ALL CAPS to prevent issues with key/value mapping
df_demo = df_demo.applymap(lambda s: s.upper() if type(s) == str else s)

df_demo['school_ID'] = df_demo['school']
df_demo['school_ID'] = df_demo['school'].map(df_school_keys)

df_demo['LEA_ID'] = df_demo['LEA']
df_demo['LEA_ID'] = df_demo['LEA'].map(df_LEA_keys)

df_demo.head()

In [ ]:
df_demo_agg = df_demo.drop('school', axis=1).groupby(['LEA', 'year', 'LEA_ID', 'type']).sum().reset_index()
df_demo_agg = df_demo_agg[df_demo_agg['LEA'].str.contains('DISTRICT')]
df_demo_agg['school'] = df_demo_agg['LEA']
df_demo_agg['school_ID'] =  df_demo_agg['school'].map(df_school_keys)

In [ ]:
df_demo_agg

In [ ]:
df_demo = df_demo.append(df_demo_agg)
df_demo = df_demo.round(0)

In [ ]:
df_demo.drop(['k', 'grade_2', 'grade_3', 'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_9', 'grade_10', 'grade_11', 'homeless', 'preschool', 'female'], axis=1, inplace=True)

In [ ]:
pct_vars = ['grade_1', 'grade_8', 'grade_12', 'male', 'american_indian', 'afam_black',
            'asian', 'hispanic', 'pacific_islander', 'white', 'econ_disadvantaged',
            'ELL', 'disabled']
for col in pct_vars:
  df_demo[col] = df_demo[col].astype(float)
  df_demo[col] = df_demo[col]/df_demo['k_12']

In [ ]:
df_demo

In [ ]:
print(df_demo['school_ID'].isnull().sum() * 100 / len(df_demo['school_ID']))
print(df_demo['LEA_ID'].isnull().sum() * 100 / len(df_demo['LEA_ID']))

In [ ]:
# merge the proficiency and financial data together my matching an observation in the financial dataset [LEA_ID, school_ID, year]
# with the same [LEA_ID, school_ID, year] in the proficiency dataset
df = df.merge(df_prof, how='left', on=['LEA_ID', 'school_ID', 'year'])

In [ ]:
df = df.merge(df_demo, how='left', on=['LEA_ID', 'school_ID', 'year'])

In [ ]:
df['amt/k12'] = (df['amt']/df['k_12']).round(2)

In [ ]:
df

In [ ]:
df[df['amt']>0].drop(['LEA_x', 'school_num', 'school_x', 'LEA_y', 'school_y', 'function', 'fund', 'type', 'stdev', 'LEA', 'school'], axis=1).groupby(['LEA_ID', 'school_ID']).head()

In [ ]:
# create 'proficiency' column that represents the average proficiency across all subjects for any school/LEA
df['proficiency'] = df[['MATHEMATICS', 'SCIENCE', 'ENGLISH LANGUAGE ARTS']].mean(axis=1)

In [ ]:
# long to wide format transformation unstacking funds
df_stack = df.groupby(['school_ID', 'program',
       'year', 'revenue', 'LEA_ID'])['amt/k12'].sum().unstack('program').reset_index()

In [ ]:
# adding back demographic variables to wide format dataframe
df_merge = df_stack.merge(df.drop(['program', 'amt', 'amt/k12', 'LEA', 'school'], axis=1), how='left', on=['school_ID', 'year', 'revenue', 'LEA_ID'])

In [ ]:
# removing observations without proficiency scores and filling rest of missing data with 0s
df_merge = df_merge[df_merge['proficiency'].notna()]
df_merge = df_merge.fillna(0)

In [ ]:
df_merge.head()

In [ ]:
# export the data to a csv
df_merge.to_csv("/content/drive/MyDrive/Clear Learning Partners/Database/royce/final.csv")